In [ ]:
!pip install ultralytics
!pip install -U ipywidgets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Define paths for input images and output labels
base_path = "/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images"  
output_labels_path = "/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/labels"

# Folders to process
folders = ["train", "valid", "test"]
categories = {"Real Human": 0, "Edited Images": 1,"AI human 2":2}

In [ ]:
os.makedirs(output_labels_path, exist_ok=True)

def generate_labels():
    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        label_folder_path = os.path.join(output_labels_path, folder)
        os.makedirs(label_folder_path, exist_ok=True)

        for category, label in categories.items():
            category_path = os.path.join(folder_path, category)

            if os.path.exists(category_path):

                category_label_path = os.path.join(label_folder_path, category)
                os.makedirs(category_label_path, exist_ok=True)

                for filename in os.listdir(category_path):
                    if filename.lower().endswith((".png", ".jpg", ".jpeg",".webp")):

                        label_file_path = os.path.join(
                            category_label_path, os.path.splitext(filename)[0] + ".txt"
                        )


                        with open(label_file_path, "w") as label_file:
                            label_file.write(f"{label}\n")

                        print(f"Label for {filename} written to {label_file_path}")
            else:
                print(f"Category folder does not exist: {category_path}")

generate_labels()

In [ ]:
import os

# Base path
base_path = "/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images"  

def get_distinct_image_types(base_path):
    # Set to store unique file extensions
    image_types = set()

    # Walk through the directory and gather extensions
    for root, dirs, files in os.walk(base_path):
        for file in files:
            # Get the file extension
            ext = os.path.splitext(file)[1].lower()  # Convert to lowercase for consistency
            if ext:  # Ensure the file has an extension
                image_types.add(ext)

    return image_types

# Get distinct image types
distinct_image_types = get_distinct_image_types(base_path)

# Print results
print("Distinct image types found:")
for img_type in distinct_image_types:
    print(img_type)


In [ ]:
# Import necessary libraries
from ultralytics import YOLO

# Load the YOLOv8 classification model
model = YOLO('yolov8s-cls.pt')

# Train the model with the desired settings
model.train(data='/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images', epochs=100, batch=16, imgsz=640,
               optimizer='Adam', lr0=0.0001,
               save=True, project='runs/train', name='exp', exist_ok=True)


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/RP/Models/best.pt")

In [ ]:
model.names

In [ ]:
results = model("/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images/test/AI human 2/6911524a-df7e-4518-9b5a-aa3546dc72fe-1.jpg")

In [ ]:
# Access prediction probabilities
probs = results[0].probs  # Extract probabilities for the first image in the batch
print(probs)
# Get the class with the highest probability
predicted_class_id = probs.top1  # Class ID with the highest probability
confidence = probs.data[predicted_class_id]  # Extract numerical confidence score
label = model.names[predicted_class_id]  # Get the class label

print(f"Predicted Class: {label}, Class ID: {predicted_class_id}, Confidence: {confidence:.2f}")

In [ ]:
for i, prob in enumerate(probs.data):
    print(f"Class {i} ({model.names[i]}): {prob:.2f}")

In [ ]:
import os

# Define the dataset directory paths
train_dir = '/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images/train'
val_dir = '/content/drive/MyDrive/RP/DataSets/imageManipulationDetection/imageManipulationDetection/images/val'

def count_files_in_classes(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts

# Count training and validation data
train_counts = count_files_in_classes(train_dir)
val_counts = count_files_in_classes(val_dir)

print("Training Set Class Distribution:", train_counts)
print("Validation Set Class Distribution:", val_counts)
